# Build The Neural Network
Neural networks comprise of layers/modules that perfom operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training
We want to be able to train our model on a hardware accelarator like the GPU or MPS, if available.
Let's check to see if `torch.cuda` or `torch.backend.mps` are available, otherwise we use the CPU.

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cuda device


## Define the Class
We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`.
Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [5]:
class NerualNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure.

In [6]:
model = NerualNetwork().to(device)
print(model)

NerualNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's `forward`, along with some background opertaions. 

**Do not call `model.forward()` directly!**

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


## Model Layers
Let's break down the layers in the FashionMNIST model. To ilustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixels values (the minibatch dimension (at dim=0) is maintained).

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
Non-linear activations are what create the complex mappings between the model. 
They are applied after linear transformations to introduce *nonlinearity*,
helping meural networks learn a wide variety of phenomena.

In this model, we use `nn.ReLU` between our linear layers, but there's other activations to introduce non-linearty in your model.

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1333,  0.5570,  0.1749,  0.0954,  0.1776, -0.4232, -0.1072, -0.0096,
          0.1434,  0.0102,  0.0606,  0.0995, -0.2699, -0.1990, -0.0240, -0.0900,
          0.0054,  0.2874, -0.1586,  0.0786],
        [ 0.1564,  0.7434,  0.0031,  0.3695, -0.2176, -0.0351,  0.0365, -0.1808,
         -0.1628, -0.0711,  0.3490, -0.2261, -0.6319,  0.1112, -0.0348, -0.1537,
          0.2370,  0.2281, -0.2680,  0.2161],
        [ 0.1998,  0.6634, -0.3179, -0.0254, -0.0008, -0.3045,  0.1139,  0.0560,
         -0.2458, -0.1296, -0.2362,  0.1276, -0.2468,  0.0323, -0.1203,  0.4637,
         -0.1516,  0.2138, -0.4518,  0.3995]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1333, 0.5570, 0.1749, 0.0954, 0.1776, 0.0000, 0.0000, 0.0000, 0.1434,
         0.0102, 0.0606, 0.0995, 0.0000, 0.0000, 0.0000, 0.0000, 0.0054, 0.2874,
         0.0000, 0.0786],
        [0.1564, 0.7434, 0.0031, 0.3695, 0.0000, 0.0000, 0.0365, 0.0000, 0.0000,
         0.0000, 0.3490, 0.0000, 0.0000, 0.1112, 0.00

### nn.Sequential
`nn.Sequential` is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax
The last linear layer of the neural network returns *logits* - raw values in [-infinity, infinity] - which are passed to the `nn.Softmax` module. The logits are scaled to values [0,1] representing the model's predicted probabilities for each class. `dim` parameter indicates the dimension along which the values must sum to 1.

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters
Many layers inside a neural network are parametrized, i.e. have associated weigths and biases that are during training.
Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters()` methods.

In this example,we iterate over each parameter, and print its size and a preview of its values.

In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NerualNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0328,  0.0199,  0.0068,  ...,  0.0197,  0.0076,  0.0186],
        [ 0.0304,  0.0301, -0.0103,  ...,  0.0283,  0.0290,  0.0260]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0054, -0.0197], device='cuda:0', grad_fn=<SliceBackward0>) 

layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0431,  0.0321,  0.0046,  ...,  0.0270, -0.0255, -0.0319],
        [-0.0016, -0.0035,  0.0373,  ...,  0.0401, -0.0333,  0.0069]],
       device='cuda:0', grad_fn=<Slice